<a href="https://colab.research.google.com/github/adityaras/Aditya_2018273_RL-M2020/blob/master/HW2/RL_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [154]:
# The Following Libraries have been imported
#   1. Numpy: Used throughout the file, for random numbers, solving equations, 
#      and initializing 2-D Matrices
#   2. Copy: To deepcopy values (In Iterative Algorithms)

import numpy as np
import copy

##Question 2: Gridworld Problem - Solving a System of Linear Equations

The function **next_step** takes in two arguments
  1.   state: The state argument is of the form (i,j) where i and j represents the row and column respectively.
  2.   action: The action argument is also a tuple which can be one of the four allowed actions in the problem i.e. (North, South, East and West).

According to the conditions i.e. taking care of out of bound conditions, and the special states given A and B, the next state S' and also the corresponding reward is returned.

In [155]:
def next_step(state, action):
  if state == (0,1):
    return (4,1), 10
  elif state == (0,3):
    return (2,3), 5
  else:
    if ((state[0]+action[0])>=0 and (state[0]+action[0])<=4 and (state[1]+action[1])>=0 and (state[1]+action[1])<=4):
      return (state[0]+action[0],state[1]+action[1]), 0
    else:
      return state, -1

The function **gridworld_prob_v_pi_s** sets up the coefficient matrix and constant matrix for the system of equations $v_{\pi}(s)$ and $v_{\pi}(s')$. The system of equations in represented by:
  
*   $v_{\pi}(s) = \Sigma_{a} \pi(a|s) [\Sigma_{s',r} p(s',r|s,a)[ r + \gamma v_{\pi}(s')]]$

The Coefficient Matrix is initialised as (states $ \times $ states) identity matrix. The Constant Matrix is initialised as a Zero Matrix of size (states $ \times $ 1) .





In [156]:
def gridworld_prob_v_pi_s(gamma, grid_rows, grid_cols, actions, pi):
  states = int(grid_rows * grid_cols)
  coeff_matrix = np.identity(states)
  const_matrix = np.zeros((states,1))
  curr_state = -1
  for i in range(grid_rows):
    for j in range(grid_cols):
      curr_state += 1
      for a in actions:
        s_dash, r = next_step((i,j),a)
        coeff_matrix[curr_state][s_dash[1]+(grid_rows*s_dash[0])] -= pi*gamma
        const_matrix[curr_state] += pi*r
  v_pi_s = np.linalg.solve(coeff_matrix, const_matrix)
  v_pi_s = v_pi_s.reshape((grid_rows,grid_cols))

  return v_pi_s

In [157]:
gamma = 0.9
grid_rows = 5
grid_cols = 5
actions = [(1,0),(0,1),(-1,0),(0,-1)]
pi = 0.25
v_pi_s = gridworld_prob_v_pi_s(gamma, grid_rows, grid_cols, actions, pi)

In [158]:
print((v_pi_s))

[[ 3.30899634  8.78929186  4.42761918  5.32236759  1.49217876]
 [ 1.52158807  2.99231786  2.25013995  1.9075717   0.54740271]
 [ 0.05082249  0.73817059  0.67311326  0.35818621 -0.40314114]
 [-0.9735923  -0.43549543 -0.35488227 -0.58560509 -1.18307508]
 [-1.85770055 -1.34523126 -1.22926726 -1.42291815 -1.97517905]]


## Question 4: Optimal State Value and Policy Functions - Solving a System of Non-Linear Equations


The System of Non Linear Equations in the example is approximated using Value Iteration. A threshold value, Theta ( $\theta$ ) set to 1e-4 which acts as a stopping criterion for the iteration process.

The Optiml Policy Function takes into account that there might be more than one optimal action and the final optimal Policy is created with keeping this is mind. For every action the action-value function $q(s,a)$ is calculated. From these $q(s,a)$ values $a_{*}$ is calculated for each state.

In [123]:
def grid_world_problem_optimal(gamma, grid_rows, grid_cols, actions, theta):
  v_s_i = np.zeros((grid_cols,grid_rows))
  v_s_i1 = np.zeros((grid_cols,grid_rows))
  curr_state = -1
  while True:
    for i in range(grid_rows):
      for j in range(grid_cols):
        max_v_s = []
        for a in actions:
          s_dash, r = next_step((i,j),a)
          v_s = r + gamma*v_s_i[s_dash[0]][s_dash[1]]
          max_v_s.append(v_s)
        v_s_i1[i][j] = max(max_v_s) 
    delta = abs(v_s_i1 - v_s_i).max()
    # print(delta)
    if delta<theta:
      return v_s_i1
    else:
      v_s_i = copy.deepcopy(v_s_i1)

In [ ]:
gamma = 0.9
grid_rows = 5
grid_cols = 5
actions = [(1,0),(0,1),(-1,0),(0,-1)]
theta = 0.00001
v_star = grid_world_problem_optimal(gamma, grid_rows, grid_cols, actions, theta)

In [ ]:
print(v_star)

[[21.97747068 24.41941186 21.97747068 19.41941186 17.47747068]
 [19.77972361 21.97747068 19.77972361 17.80174304 16.02156873]
 [17.80174304 19.77972361 17.80174304 16.02156873 14.41941186]
 [16.02156873 17.80174304 16.02156873 14.41941186 12.97747068]
 [14.41941186 16.02156873 14.41941186 12.97747068 11.67972361]]


In [3]:
def grid_world_problem_optimal_policy(actions, grid_rows, grid_cols,v_star,dir):
  state = -1
  for i in range(grid_rows):
      for j in range(grid_cols):
        state += 1
        q_s_a = np.zeros(len(actions))
        k = 0
        for a in actions:
            s_dash, r = next_step((i,j), a)
            q_s_a[k] = r + (gamma * v_star[s_dash[0]][s_dash[1]])
            k += 1
        a_star = max(q_s_a)
        s = ""
        for q in range(len(q_s_a)):
          if a_star == q_s_a[q]:
            s += dir[q] + "+"
        s = s[:-1]
        print(s, end="")
        if (state+1)%5 == 0:
          print('\n')
        else:
          print("; ", end ="") 

In [ ]:
gamma = 0.9
grid_rows = 5
grid_cols = 5
actions = [(1,0),(0,1),(-1,0),(0,-1)]
dir = {0:"South", 1:"East", 2:"North", 3:"West"}
grid_world_problem_optimal_policy(actions, grid_rows, grid_cols,v_star,dir)

East; South+East+North+West; West; South+East+North+West; West

East+North; North; North+West; West; West

East+North; North; North+West; North+West; North+West

East+North; North; North+West; North+West; North+West

East+North; North; North+West; North+West; North+West



## Question 6: Policy and Value Iteration in the Gridworld Example 4.1

### Policy Iteration

Firstly a new **next_step_ex41** is implemented according to the new Grid World Conditions, which is very similar to the one implemented above but also takes an additional argument, *grid_rows* to take care of terminal states.

For Policy Iteration,
Two functions are implemented: Followed Instructions provided in the Textbook (Sutton)


1.   Policy Evaluation
2.   Policy Improvement

The Bug mentioned in Exercise 4.4 will not take place because we take all possible maximum values and update them simultaneously. As a result the bug that the policy may oscillate between two maximum values.


In [30]:
def next_step_ex41(state, action, grid_rows):
  if ((state[0]+action[0])>=0 and (state[0]+action[0])<4 and (state[1]+action[1])>=0 and (state[1]+action[1])<4):
    if (state[0]+action[0], state[1]+action[1]) == (0,0) or (state[0]+action[0], state[1]+action[1]) == (grid_rows-1, grid_rows-1):
      return (grid_rows-1,grid_cols-1), -1
    else:
      return (state[0]+action[0],state[1]+action[1]), -1
  else:
    return state, -1

In [109]:
def grid_world_4x4_policy_eval(v_pi, grid_rows, grid_cols, actions, pi,term_state, theta):
  # v_pi = np.zeros((grid_rows,grid_cols))
  while(True):
    delta = 0
    for i in range(grid_rows):
      for j in range(grid_cols):
        if (i,j) not in term_state:
          v = copy.deepcopy(v_pi[i][j])
          val = 0
          for a in actions:
            s_dash, r = next_step_ex41((i,j), a, grid_rows)
            val += pi * (r + v_pi[s_dash[0]][s_dash[1]])
          v_pi[i][j] = val
          delta = max(delta, abs(v_pi[i][j] - v))
          # print("Delta "+str(delta))
    if delta < theta:
      return v_pi
    

In [88]:
grid_rows = 4
grid_cols = 4
v_pi = np.zeros((grid_rows,grid_cols))
actions = [(1,0),(0,1),(-1,0),(0,-1)]
pi = 0.25
theta = 0.00001
term_state = [(0,0),(grid_rows-1,grid_cols-1)]
dir = {0:"South", 1:"East", 2:"North", 3:"West"}
v_pi = grid_world_4x4_policy_eval(v_pi,grid_rows, grid_cols, actions, pi,term_state, theta)

In [ ]:
print(v_pi)

[[  0.         -13.99993529 -19.99990698 -21.99989761]
 [-13.99993529 -17.9999206  -19.99991379 -19.99991477]
 [-19.99990698 -19.99991379 -17.99992725 -13.99994569]
 [-21.99989761 -19.99991477 -13.99994569   0.        ]]


In [110]:
def grid_world_4x4_policy_improv(pi_mat,v_pi,grid_rows, grid_cols, actions, pi,term_state, action_map):
  no_actions = len(actions)
  # pi_mat = np.full((grid_rows,grid_cols,no_actions),pi)
  policy_stable = True
  for i in range(grid_rows):
    for j in range(grid_cols):
      if (i,j) not in term_state:
        old_a = copy.deepcopy(pi_mat[i][j])
        a_ind = 0
        returns = []
        for a in actions:
          s_dash, r = next_step_ex41((i,j), a, grid_rows)
          returns.append(r + v_pi[s_dash[0]][s_dash[1]])
        max_val = max(returns)
        max_a = []
        for  ret in range(len(returns)):
          if returns[ret] == max_val:
            max_a.append(ret)
        pi_mat[i][j] = [0.] * no_actions
        for a_ind in max_a:
          pi_mat[i][j][a_ind] = (1/len(max_a))
        for ind in range(len(old_a)):
          if (old_a[ind] != pi_mat[i][j][ind]):
            policy_stable = False
            break
  return pi_mat, policy_stable          

In [97]:
grid_rows = 4
grid_cols = 4
actions = [(1,0),(0,1),(-1,0),(0,-1)]
action_map = {0:(1,0),1:(0,1),2:(-1,0),3:(0,-1)}
pi = 0.25
term_state = [(0,0),(grid_rows-1,grid_cols-1)]
dir = {0:"D", 1:"R", 2:"U", 3:"L"}
no_actions = len(actions)
pi_mat = np.full((grid_rows,grid_cols,no_actions),pi)
print(grid_world_4x4_policy_improv(pi_mat,v_pi,grid_rows, grid_cols, actions, pi,term_state, action_map))

(array([[[0.25, 0.25, 0.25, 0.25],
        [0.  , 0.  , 0.  , 1.  ],
        [0.  , 0.  , 0.  , 1.  ],
        [0.  , 0.  , 0.  , 1.  ]],

       [[0.  , 0.  , 1.  , 0.  ],
        [0.  , 0.  , 0.5 , 0.5 ],
        [0.  , 0.  , 0.  , 1.  ],
        [1.  , 0.  , 0.  , 0.  ]],

       [[0.  , 0.  , 1.  , 0.  ],
        [0.  , 0.  , 1.  , 0.  ],
        [0.  , 1.  , 0.  , 0.  ],
        [1.  , 0.  , 0.  , 0.  ]],

       [[0.  , 0.  , 1.  , 0.  ],
        [0.  , 1.  , 0.  , 0.  ],
        [0.  , 1.  , 0.  , 0.  ],
        [0.25, 0.25, 0.25, 0.25]]]), False)


In [118]:
def fin_policy(pi_mat,dir,grid_rows,grid_cols,term_state):
  for i in range(grid_rows):
    for j in range(grid_cols):
      if (i,j) not in term_state:
        val = max(pi_mat[i][j])
        s=""
        for k in range(len(pi_mat[i][j])):
          if val == pi_mat[i][j][k]:
            s+=dir[k]
        print(s,end="")
      else:
        print("#",end="")
      if (j==grid_cols-1):
        print("\n")
      else:
        print(";",end=" ")

In [120]:
def grid_4x4_PI(grid_rows, grid_cols, actions, pi,term_state, theta, action_map,dir):
  i = 0
  v_pi = np.zeros((grid_rows,grid_cols))
  no_actions = len(actions)
  pi_mat = np.full((grid_rows,grid_cols,no_actions),pi)
  while True:
    print("Iteration "+str(i))
    print("Evaluating Policy...")

    v = grid_world_4x4_policy_eval(v_pi, grid_rows, grid_cols, actions, pi,term_state, theta)
    v_pi = copy.deepcopy(v)
    print(v_pi)

    print("Improving Policy...")

    pi_mat_temp, stable = grid_world_4x4_policy_improv(pi_mat,v_pi,grid_rows, grid_cols, actions, pi,term_state, action_map)
    pi_mat = copy.deepcopy(pi_mat_temp)
    print(pi_mat)

    if stable:
      print("\nPolicy Converged\n")
      fin_policy(pi_mat,dir,grid_rows,grid_cols,term_state)
      break
    i+=1

In [121]:
grid_rows = 4
grid_cols = 4
actions = [(1,0),(0,1),(-1,0),(0,-1)]
action_map = {0:(1,0),1:(0,1),2:(-1,0),3:(0,-1)}
pi = 0.25
term_state = [(0,0),(grid_rows-1,grid_cols-1)]
dir = {0:"D", 1:"R", 2:"U", 3:"L"}
theta = 0.00001
grid_4x4_PI(grid_rows, grid_cols, actions, pi,term_state, theta, action_map,dir)

Iteration 0
Evaluating Policy...
[[  0.         -13.99993529 -19.99990698 -21.99989761]
 [-13.99993529 -17.9999206  -19.99991379 -19.99991477]
 [-19.99990698 -19.99991379 -17.99992725 -13.99994569]
 [-21.99989761 -19.99991477 -13.99994569   0.        ]]
Improving Policy...
[[[0.25 0.25 0.25 0.25]
  [0.   0.   0.   1.  ]
  [0.   0.   0.   1.  ]
  [0.   0.   0.   1.  ]]

 [[0.   0.   1.   0.  ]
  [0.   0.   0.5  0.5 ]
  [0.   0.   0.   1.  ]
  [1.   0.   0.   0.  ]]

 [[0.   0.   1.   0.  ]
  [0.   0.   1.   0.  ]
  [0.   1.   0.   0.  ]
  [1.   0.   0.   0.  ]]

 [[0.   0.   1.   0.  ]
  [0.   1.   0.   0.  ]
  [0.   1.   0.   0.  ]
  [0.25 0.25 0.25 0.25]]]
Iteration 1
Evaluating Policy...
[[  0.         -13.99994072 -19.99991477 -21.99990619]
 [-13.99994072 -17.99992725 -19.99992101 -19.99992192]
 [-19.99991477 -19.99992101 -17.99993335 -13.99995024]
 [-21.99990619 -19.99992192 -13.99995024   0.        ]]
Improving Policy...
[[[0.25 0.25 0.25 0.25]
  [0.   0.   0.   1.  ]
  [0.   0.  

### Value Iteration

A similar process to as followed in Question 4 is repeated with a few tweaks to fit the gridworld in Ex 4.1. 

In [148]:
def grid_4x4_value_VI(v_s, grid_rows, grid_cols, actions,term_state, theta):
  cnt = 0
  while (True): 
    print("Iteration "+str(cnt))
    delta = 0
    for i in range(grid_rows):
      for j in range(grid_cols):
        if (i,j) not in term_state:
          old_v = copy.deepcopy(v_s[i][j])
          # val = -sys.max - 1
          returns = []
          for a in actions:
            s_dash, r = next_step_ex41((i,j), a, grid_rows)
            returns.append(r + v_s[s_dash[0]][s_dash[1]])
          v_s[i][j] = max(returns)
          delta = max(delta, abs(old_v - v_s[i][j]))
    print(v_s)
    cnt+=1
    if delta<theta:
      return v_s    

In [149]:
grid_rows = 4
grid_cols = 4
v_s = np.zeros((grid_rows, grid_cols))
actions = [(1,0),(0,1),(-1,0),(0,-1)]
theta = 0.00001
term_state = [(0,0),(grid_rows-1,grid_cols-1)]
v_star = grid_4x4_value_VI(v_s, grid_rows, grid_cols, actions,term_state, theta)
# print(v_star)

Iteration 0
[[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1.  0.]]
Iteration 1
[[ 0. -1. -2. -2.]
 [-1. -2. -2. -2.]
 [-2. -2. -2. -1.]
 [-2. -2. -1.  0.]]
Iteration 2
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]
Iteration 3
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]


In [152]:
def grid_4x4_policy_VI(v_star, pi_s, grid_rows, grid_cols, actions,term_state,dir):
  for i in range(grid_rows):
    for j in range(grid_cols):
      if (i,j) not in term_state:
        returns = []
        for a in actions:
          s_dash, r = next_step_ex41((i,j), a, grid_rows)
          returns.append(r + v_star[s_dash[0]][s_dash[1]])
        val = max(returns)
        for k in range(len(returns)):
          if val == returns[k]:
              pi_s[i][j][k] = 1
  print(pi_s)
  fin_policy(pi_s,dir,grid_rows,grid_cols,term_state)   

In [153]:
grid_rows = 4
grid_cols = 4
actions = [(1,0),(0,1),(-1,0),(0,-1)]
no_actions = len(actions)
pi_s = np.zeros((grid_rows, grid_cols,no_actions))
dir = {0:"D", 1:"R", 2:"U", 3:"L"}
term_state = [(0,0),(grid_rows-1,grid_cols-1)]
grid_4x4_policy_VI(v_star, pi_s, grid_rows, grid_cols, actions,term_state,dir)

[[[0. 0. 0. 0.]
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [1. 0. 0. 1.]]

 [[0. 0. 1. 0.]
  [0. 0. 1. 1.]
  [1. 1. 1. 1.]
  [1. 0. 0. 0.]]

 [[0. 0. 1. 0.]
  [1. 1. 1. 1.]
  [1. 1. 0. 0.]
  [1. 0. 0. 0.]]

 [[0. 1. 1. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 0. 0.]]]
#; L; L; DL

U; UL; DRUL; D

U; DRUL; DR; D

RU; R; R; #



## Question 7: Policy Iteration, Book Exercise 4.7